<a href="https://colab.research.google.com/github/oroobyaseen/coursera.exercises/blob/gh-pages/Copy_of_Untitled39.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
# re-size all the images to this
IMAGE_SIZE = [228, 228]
train_path="/content/drive/MyDrive/New folder (6)/train"
test_path="/content/drive/MyDrive/New folder (6)/test"
val_path="/content/drive/MyDrive/New folder (6)/val"

In [3]:
x_train=[]

for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_train.append(img_arr)
x_test=[]

for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_test.append(img_arr)
    
x_val=[]

for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_val.append(img_arr)
    

In [37]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [38]:
train_x.shape,test_x.shape,val_x.shape

((2450, 228, 228, 3), (310, 228, 228, 3), (305, 228, 228, 3))

In [39]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#train_datagen = ImageDataGenerator(rescale = 1./255,
 #                                   shear_range = 0.2,
  #                                  zoom_range = 0.2,
   #                                 horizontal_flip = True)

train_datagen = ImageDataGenerator( rescale = 1./255,
                                     
   # rescale = 1./255,
                                   shear_range=0.2,
                                   #zoom_range=0.2,
                                  horizontal_flip=True,
                                   #validation_split = 0.
                                       #rotation_range=20,
                                #width_shift_range=0.1,
                                #height_shift_range=0.1,
                                #shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               # horizontal_flip=False,
                             vertical_flip= True,
                               # fill_mode='constant',
                                #cval=0,
                                   )
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (228, 228),
                                                 batch_size = 128,
                                                 class_mode = 'sparse')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (228, 228),
                                            batch_size = 128,
                                            class_mode = 'sparse')

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (228, 228),
                                            batch_size = 128,
                                            class_mode = 'sparse')

Found 2450 images belonging to 3 classes.
Found 310 images belonging to 3 classes.
Found 305 images belonging to 3 classes.


In [48]:
training_set.class_indices

{'Glioma': 0, 'Meningioma': 1, 'Pituitary tumor': 2}

In [49]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes
train_y.shape,test_y.shape,val_y.shape

((2450,), (310,), (305,))

In [50]:
# add preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False
# our layers - you can add more if you want
x = Flatten()(vgg.output)

prediction = Dense(3, activation='softmax')(x)

In [51]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 228, 228, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 228, 228, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 228, 228, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 114, 114, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 114, 114, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 114, 114, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 57, 57, 128)       0   

In [52]:
# tell the model what cost and optimization method to use
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

#Early stopping to avoid overfitting of model
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=20,
  #callbacks=[early_stop],
  batch_size=128,shuffle=True)

In [ ]:
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()

plt.savefig('vgg-loss-rps-1.png')
plt.show()

In [ ]:
# accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()

plt.savefig('vgg-acc-rps-1.png')
plt.show()

In [ ]:
model.evaluate(test_x,test_y,batch_size=128)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)
y_score = accuracy_score(y_pred,test_y)

In [ ]:
print(classification_report(y_pred,test_y))

In [ ]:
confusion_matrix(y_pred,test_y)

In [ ]:
from sklearn.metrics import roc_auc_score



In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:

from sklearn.metrics import roc_auc_score

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save("vgg-16-rps-final.h5")

In [ ]:
test_model = load_model('/content/vgg-16-rps-final.h5')
predictions = test_model.predict(test_set, steps=1, verbose=0)

In [ ]:
predictions

In [ ]:
rounded_prediction = np.argmax(predictions, axis=-1)
for i in rounded_prediction:
    print(i)

In [ ]:
import math
batch_size = 128